# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')

df = pd.read_csv('homepage_actions.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df['id'].value_counts()

750053    2
389447    2
283040    2
918794    2
208714    2
         ..
467818    1
629924    1
820336    1
424916    1
461199    1
Name: id, Length: 6328, dtype: int64

6,328 unique people

In [5]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

6,328 people viewed and 1,860 clicked. Everyone viewed.

In [6]:
#separating between views and clicks by id
click_id = set(df[df['action']== 'click']['id'].unique())
view_id = set(df[df['action']== 'view']['id'].unique())

print("Number of Clickers Who Did Not View: ", len(click_id - view_id))
print("Number of Viewers Who Did Not Click: ", len(view_id - click_id))

Number of Clickers Who Did Not View:  0
Number of Viewers Who Did Not Click:  4468


Everyone who clicked also viewed. 4,468 people viewed but did not click. No anomalies.

In [7]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [8]:
exp_id = set(df[df['group'] == 'experiment']['id'].unique())
con_id = set(df[df['group'] == 'control']['id'].unique())
len(exp_id & con_id)

0

There's no overlap with the Experiment and Control groups. With no anomalies or overlap with the groups, the data doesn't need to be cleaned.

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [9]:
df['count'] = 1
#assigning a value for Yes for every "click" and "view"

In [10]:
#assigning a value for No for every "click" and "view" (0 for No and 1 for Yes)
#separating the control and experiment groups
exp = df[df['group'] == 'experiment'].pivot(index = 'id', columns = 'action', values = 'count').fillna(value=0)
con = df[df['group'] == 'control'].pivot(index = 'id', columns = 'action', values = 'count').fillna(value=0)

print("Total Users: \t\t\tExperiment: {} users \t\tControl: {} users".format(len(exp), len(con)))
print("Total Clicks: \t\t\tExperiment: {} users \t\tControl: {} users".format(int(exp['click'].sum()), int(con['click'].sum())))
print("% of Users who Clicked: \tExperiment: {} users \tControl: {} users".format(round(exp['click'].mean(), 4), round(con['click'].mean(), 4)))

Total Users: 			Experiment: 2996 users 		Control: 3332 users
Total Clicks: 			Experiment: 928 users 		Control: 932 users
% of Users who Clicked: 	Experiment: 0.3097 users 	Control: 0.2797 users


In [18]:
import scipy.stats as stats

#Welch's t statistic
def welch_t(a, b):
    num = a.mean() - b.mean()
    den = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    return np.abs(num/den)

#Degrees of Freedom
def welch_df(a, b):
    s1 = a.var(ddof=1)
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    num = (s1/n1 + s2/n2)**2
    den = (s1/n1)**2/(n1 - 1) + (s2/n2)**2/(n2 -1)
    
    return num/den

#p-value for Welch t-test    
def p_value_welch_ttest(a, b, two_sided=False):
    t = welch_t(a, b)
    df = welch_df(a, b)
    p = 1-stats.t.cdf(np.abs(t), df)
    if two_sided:
        return 2*p
    else:
        return p

In [19]:
print("p-value of Welch's t-test: " , p_value_welch_ttest(exp['click'], con['click']))

p-value of Welch's t-test:  0.004466402814337078


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [23]:
con_click_rate = con['click'].mean()
expected_exp_clicks = con_click_rate * len(exp)
print("Expected Clicks for Experiment Group:", round(expected_exp_clicks, 2), "clicks")

Expected Clicks for Experiment Group: 838.02 clicks


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [28]:
#total number in experiment group
n = len(exp)
#probability of clicking in control group
p = con_click_rate
var = n*p*(1-p)
std = np.sqrt(var)

print("Standard Deviation:", round(std, 4), "clicks")

actual_exp_clicks = exp['click'].sum()
z_score = (actual_exp_clicks - expected_exp_clicks)/std
print("Z-Score:", round(z_score, 4), "clicks")

Standard Deviation: 24.5685 clicks
Z-Score: 3.6625 clicks


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [29]:
p_value = stats.norm.sf(z_score)
print("p-value:", p_value)

p-value: 0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: It does match the previous statistical test. The p-value is lower but both numbers are significantly lower than 0.05, so both tests can reject the null hypothesis. The experimental is more effective than the control. The experimental website generated 928 clicks and was expected to generate about 838 clicks from the control website data.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.